In [4]:
%matplotlib qtagg

import numpy as np
from scipy import constants
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from qutip import * 
import qutip as qt

In [5]:
# params
omega_q = 5.0
omega_d = 5.0
A = 0.063
phi = 0
alpha = 1
beta = 1

# normalize and define psi_0
norm = np.sqrt(alpha * alpha + beta * beta)
alpha = alpha / norm
beta = beta / norm
psi_0 = alpha * basis(2, 0) + beta * basis(2, 1)

# substitution variables
Omega = A * np.cos(phi) + A * 1j * np.sin(phi) # Omega = e^(i * phi)

# Hamiltonians and time dependent functions
def driving(t, args):
    return  A * np.cos(omega_d * t + phi)

def counter_plus(t, args):
    return np.cos(2 * omega_d * t) + 1j * np.sin(2 * omega_d * t)

def counter_minus(t, args):
    return np.cos(2 * omega_d * t) - 1j * np.sin(2 * omega_d * t)

def stark(t, args):
    return 0.1 * np.cos(omega_d * t)

H0 = (omega_q * sigmaz()) / 2
H1 = sigmax()
H_s = sigmaz() # stark shift 
H_lab = [H0, [H1, driving]] # lab frame hamiltonian
H_lab_stark = [H0, [H1, driving], [H_s, stark]] # lab frame hamiltonian with stark shift

H_rwa = (1/2) * Omega * create(2) + (1/2) * np.conj(Omega) * destroy(2) # RWA hamiltonian in qubit frame (rotating frame)

H_counter_plus = (1/2) * Omega * create(2)
H_counter_minus = (1/2) * np.conj(Omega) * destroy(2)
H_qubit = [H_rwa, [H_counter_plus, counter_plus], [H_counter_minus, counter_minus]] # hamiltonian in qubit frame with counter-rotating terms

H_offres = [(omega_q - omega_d) * sigmaz(), H_rwa, [H_counter_plus, counter_plus], [H_counter_minus, counter_minus]] # off-resonant hamiltonian
H_stark = [H_rwa, [H_counter_plus, counter_plus], [H_counter_minus, counter_minus], H_s]
# time variable
t_list = np.linspace(0, 100, 1000)

In [6]:
def plot_bloch(Hamiltonian):
    fig = plt.figure(figsize=(12, 8))
    gs = GridSpec(3, 3, figure=fig, hspace=0.6, wspace=0.5)
    
    ax = fig.add_subplot(gs[:, :2], projection='3d')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
 
    result = sesolve(Hamiltonian, psi_0, t_list)
    result_expect = sesolve(Hamiltonian, psi_0, t_list, [sigmax(), sigmay(), sigmaz()])
    
    alpha = [result.states[i][0] for i in range(len(result.states))]
    beta = [result.states[i][1] for i in range(len(result.states))]

    x = 2 * np.real(np.conjugate(alpha) * beta)
    y = 2 * np.imag(np.conjugate(alpha) * beta)
    z = alpha * np.conjugate(alpha) - beta * np.conjugate(beta)

    ax.plot(x, y, z, label='qubit')

    phi, theta = np.linspace(0, 2 * np.pi, 50), np.linspace(0, np.pi, 50)
    phi, theta = np.meshgrid(phi, theta)
    sp_x = np.sin(theta) * np.cos(phi)
    sp_y = np.sin(theta) * np.sin(phi)
    sp_z = np.cos(theta)

    ax.plot_wireframe(sp_x, sp_y, sp_z, rstride=4, cstride=4, color='#d3d3d3', edgecolor='k', alpha=0.3)
    ax.text(0, 0, 1.2, '|0⟩')
    ax.text(0, 0, -1.2, '|1⟩')
    
    ax_x = fig.add_subplot(gs[0, 2:])
    ax_x.plot(t_list, result_expect.expect[0], label='X')
    ax_x.set_title("⟨X⟩")
    ax_x.set_xlabel("Time (ns)")
    ax_x.set_ylabel("x")
    ax_y = fig.add_subplot(gs[1, 2:])
    ax_y.plot(t_list, result_expect.expect[1], label='Y')
    ax_y.set_title("⟨Y⟩")
    ax_y.set_xlabel("Time (ns)")
    ax_y.set_ylabel("y")
    ax_z = fig.add_subplot(gs[2, 2:])
    ax_z.plot(t_list, result_expect.expect[2], label='Z')
    ax_z.set_title("⟨Z⟩")
    ax_z.set_xlabel("Time (ns)")
    ax_z.set_ylabel("z")

    ax.set_title("Qubit Bloch Sphere")
    plt.show()

plot_bloch(H_offres)

In [96]:
def plot_bloch_qutip():
    result = sesolve(H, psi_0, t_list)

    bloch = Bloch()
    bloch.add_states(result.states)
    
    bloch.make_sphere()
    plt.title("Qubit Bloch Sphere")
    plt.show()

plot_bloch_qutip()